# NeomarilPreprocessing

This notebook give a exemple on how to use Neomaril to deploy a preprocessing

## Imports

In [1]:
from neomaril_codex.preprocessing import NeomarilPreprocessingClient
from neomaril_codex.model import NeomarilModelClient

## NeomarilPreprocessingClient

In [3]:
client = NeomarilPreprocessingClient()

2023-10-16 14:38:33.106 | INFO     | neomaril_codex.preprocessing:__init__:266 - Loading .env
2023-10-16 14:38:33.108 | INFO     | neomaril_codex.base:__init__:90 - Loading .env
2023-10-16 14:38:36.131 | INFO     | neomaril_codex.base:__init__:102 - Successfully connected to Neomaril


## Creating sync pre processing

In [11]:
PATH = './samples/syncPreprocessing/'

In [14]:
sync_preprocessing = client.create('Teste preprocessing Sync', # model_name
                            'process', # name of the scoring function
                            PATH+'app.py', # Path of the source file
                            PATH+'requirements.txt', # Path of the requirements file, 
                            schema=PATH+'schema.json', # Path of the schema file, but it could be a dict (only required for Sync models)
#                           env=PATH+'.env'  #  File for env variables (this will be encrypted in the server)
#                           extra_files=[PATH+'utils.py'], # List with extra files paths that should be uploaded along (they will be all in the same folder)
                            python_version='3.9', # Can be 3.7 to 3.10
                            operation="Sync", # Can be Sync or Async
                            group='datarisk' # Model group (create one using the client)
                            )

2023-10-16 11:57:09.764 | INFO     | neomaril_codex.preprocessing:__upload_preprocessing:544 - Script was registered! - Hash: "S12cc6e9f3084b59a8d17fa309a4b0045e329a62ee7b43b28300d5bb45526f82"
2023-10-16 11:57:11.921 | INFO     | neomaril_codex.preprocessing:__host_preprocessing:575 - Preprocessing host in process - Hash: S12cc6e9f3084b59a8d17fa309a4b0045e329a62ee7b43b28300d5bb45526f82


Wating for deploy to be ready........

2023-10-16 11:58:32.883 | INFO     | neomaril_codex.preprocessing:get_preprocessing:358 - Preprocessing S12cc6e9f3084b59a8d17fa309a4b0045e329a62ee7b43b28300d5bb45526f82 its deployed. Fetching preprocessing.
2023-10-16 11:58:32.885 | INFO     | neomaril_codex.preprocessing:__init__:19 - Loading .env


In [15]:
sync_preprocessing.set_token('TODO')

2023-10-16 12:00:42.516 | INFO     | neomaril_codex.preprocessing:set_token:123 - Token for group datarisk added.


In [16]:
result = sync_preprocessing.run({'variable' : 100})
result

{'mean_radius': 0,
 'mean_texture': 0,
 'mean_perimeter': 0,
 'mean_area': 0,
 'mean_smoothness': 0,
 'mean_compactness': 0,
 'mean_concavity': 0,
 'mean_concave_points': 0,
 'mean_symmetry': 0,
 'mean_fractal_dimension': 0,
 'radius_error': 0,
 'texture_error': 0,
 'perimeter_error': 0,
 'area_error': 0,
 'smoothness_error': 0,
 'compactness_error': 0,
 'concavity_error': 0,
 'concave_points_error': 0,
 'symmetry_error': 0,
 'fractal_dimension_error': 0,
 'worst_radius': 0,
 'worst_texture': 0,
 'worst_perimeter': 0,
 'worst_area': 0,
 'worst_smoothness': 0,
 'worst_compactness': 0,
 'worst_concavity': 0,
 'worst_concave_points': 0,
 'worst_symmetry': 0,
 'worst_fractal_dimension': 0}

## Creating async pre processing

In [6]:
PATH = './samples/asyncPreprocessing/'

async_preprocessing = client.create('Teste preprocessing Async', # model_name
                            'process', # name of the scoring function
                            PATH+'app.py', # Path of the source file
                            PATH+'requirements.txt', # Path of the requirements file, 
                            # env=PATH+'.env',  #  File for env variables (this will be encrypted in the server)
                            # extra_files=[PATH+'input.csv'], # List with extra files paths that should be uploaded along (they will be all in the same folder)
                            python_version='3.9', # Can be 3.7 to 3.10
                            operation="Async", # Can be Sync or Async
                            group='datarisk', # Model group (create one using the client)
                            input_type='csv'
                            )

2023-10-16 14:39:03.428 | INFO     | neomaril_codex.preprocessing:__upload_preprocessing:544 - Script was registered! - Hash: "S7c12eeaeab5445e9268149487fdd89b0122476ede7d42e9a3524cc40ea744fa"
2023-10-16 14:39:04.845 | INFO     | neomaril_codex.preprocessing:__host_preprocessing:575 - Preprocessing host in process - Hash: S7c12eeaeab5445e9268149487fdd89b0122476ede7d42e9a3524cc40ea744fa


Wating for deploy to be ready..................

2023-10-16 14:42:19.855 | INFO     | neomaril_codex.preprocessing:get_preprocessing:358 - Preprocessing S7c12eeaeab5445e9268149487fdd89b0122476ede7d42e9a3524cc40ea744fa its deployed. Fetching preprocessing.
2023-10-16 14:42:19.857 | INFO     | neomaril_codex.preprocessing:__init__:19 - Loading .env


In [5]:
async_preprocessing.set_token('TODO')

2023-10-16 11:25:55.396 | INFO     | neomaril_codex.preprocessing:set_token:123 - Token for group datarisk added.


In [6]:
execution = async_preprocessing.run(PATH+'input.csv')

2023-10-16 11:25:55.766 | INFO     | neomaril_codex.preprocessing:run:177 - Execution '8' started to generate 'D1539f608c49485997db85ba0df2165347e048c53d7e42b88e415908893feac8'. Use the id to check its status.
2023-10-16 11:25:55.768 | INFO     | neomaril_codex.base:__init__:279 - Loading .env


In [7]:
execution.get_status()

{'ExecutionId': '8', 'Status': 'Running', 'Message': None}

In [8]:
execution.wait_ready()
execution.download_result()

2023-10-16 11:28:59.562 | INFO     | neomaril_codex.base:download_result:412 - Output saved in ./output_8.zip


## Access created pre processing

In [ ]:
client.search_preprocessing()

In [ ]:
preprocessing = client.get_preprocessing(preprocessing_id='id', group='datarisk')

In [ ]:
preprocessing.set_token('group_token')

In [ ]:
execution = preprocessing.run(PATH+'input.csv')

In [ ]:
execution.get_status()

In [ ]:
execution.download_result()

## Access created executions

In [ ]:
execution_4 = preprocessing.get_preprocessing_execution(exec_id='4')

execution_4.download_result()

## Using preprocessing with models

In [2]:
model_client = NeomarilModelClient()

2023-10-16 14:55:24.440 | INFO     | neomaril_codex.model:__init__:723 - Loading .env
2023-10-16 14:55:24.441 | INFO     | neomaril_codex.base:__init__:90 - Loading .env
2023-10-16 14:55:25.477 | INFO     | neomaril_codex.base:__init__:102 - Successfully connected to Neomaril


In [18]:
sync_model = model_client.get_model(group='datarisk', model_id='M1488299aa18492d80680857a5c1eedbcd4c53ef5b184335b32104449fee97cd')

sync_model.set_token('TODO')

sync_model.predict(data=sync_model.schema, preprocessing=sync_preprocessing)

2023-10-16 12:49:24.043 | INFO     | neomaril_codex.model:__init__:722 - Loading .env
2023-10-16 12:49:24.046 | INFO     | neomaril_codex.base:__init__:90 - Loading .env
2023-10-16 12:49:25.678 | INFO     | neomaril_codex.base:__init__:102 - Successfully connected to Neomaril
2023-10-16 12:49:27.941 | INFO     | neomaril_codex.model:get_model:820 - Model M1488299aa18492d80680857a5c1eedbcd4c53ef5b184335b32104449fee97cd its deployed. Fetching model.
2023-10-16 12:49:27.942 | INFO     | neomaril_codex.model:__init__:69 - Loading .env
2023-10-16 12:49:31.710 | INFO     | neomaril_codex.model:set_token:297 - Token for group datarisk added.


{'pred': 0, 'proba': 0.005841062869876623}

In [3]:
async_model = model_client.get_model(group='datarisk', model_id='M891d66cd91e4d3a87146884d01dbe7173ae8f7ca58c46b0a17a123a979e568f')

async_model.set_token('TODO')

PATH = './samples/asyncModel/'

execution = async_model.predict(PATH+'input.csv', preprocessing=async_preprocessing)
execution.wait_ready()

2023-10-16 14:55:28.422 | INFO     | neomaril_codex.model:get_model:821 - Model M891d66cd91e4d3a87146884d01dbe7173ae8f7ca58c46b0a17a123a979e568f its deployed. Fetching model.
2023-10-16 14:55:28.423 | INFO     | neomaril_codex.model:__init__:69 - Loading .env
2023-10-16 14:55:31.819 | INFO     | neomaril_codex.model:set_token:297 - Token for group datarisk added.
2023-10-16 14:55:31.820 | INFO     | neomaril_codex.preprocessing:set_token:123 - Token for group datarisk added.
2023-10-16 14:55:32.183 | INFO     | neomaril_codex.preprocessing:run:177 - Execution '13' started to generate 'Dc9b3d9e15a8470abdc72ed91ba9b43a15a9d28dacb44075b21f754e423a0702'. Use the id to check its status.
2023-10-16 14:55:32.184 | INFO     | neomaril_codex.base:__init__:279 - Loading .env
2023-10-16 14:58:35.384 | INFO     | neomaril_codex.base:download_result:413 - Output saved in ./result_preprocessing
2023-10-16 14:58:35.865 | INFO     | neomaril_codex.model:predict:366 - Execution '5' started. Use the id 

In [4]:
execution.download_result()

2023-10-16 15:08:31.451 | INFO     | neomaril_codex.base:download_result:413 - Output saved in ./output.zip
